# Analyze subject and object in setence demo

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk.data
import chardet
import re
import os
import json
from pathlib import Path
import spacy
import nltk.data
nlp = spacy.load('en_core_web_sm')

In [2]:
from collections import Counter
import spacy
from tabulate import tabulate
nlp = spacy.load('en_core_web_lg')
import nltk
from nltk.tokenize import sent_tokenize

In [3]:
# main function for generating pre-treated data 
def main():
    #dir = "/Users/chenjia/study/nlp/code/group_project/text_data/"
    dir = "/Users/isthatyoung/temp/NLP-Characters-Relationships/text_data/"
    raw_text_data = read_file(dir)
    valid_data = pre_processing(raw_text_data)
    return valid_data

In [4]:
# read file # same as siyang's code
def read_file(dir):
    json_file = Path('{}Raw-Harry-Potter-Series.json'.format(dir))
    book = {}
    if not json_file.exists():
        files = []
        pattern = '\.txt'
        for readfile in os.listdir(dir):
            if re.search(pattern, readfile):
                files.append(readfile)
        pattern = '(.+)\..+$'
        for file_name in files:
            decode_text = []
            with open(dir + file_name, 'rb') as f_read:
                text = f_read.readlines()
                for line in text:
                    type = chardet.detect(line)
                    line = line.decode(type["encoding"])
                    decode_text.append(line)
                file_name = re.findall(pattern, file_name)[0]
                book[file_name] = decode_text
            f_read.close()

        ##TODO:Save book to json file
        with open(dir + 'Raw-Harry-Potter-Series.json', 'a') as outfile:
            json.dump(book, outfile, ensure_ascii=False)
            outfile.write('\n')
    else:
        readfile = open('{}Raw-Harry-Potter-Series.json'.format(dir),"r")
        book = eval(readfile.read())
    return book

In [5]:
# data processing: we need each novel to be tokenized into sentences, not a whole novel, not tokens.
### 此方法断句会产生断开Mr.Weasley的错误，但也不是不能用。 当前主函数没有调用这个。###

def preprocessing(data):
    
    data_into_sentences = {}
    
    for key in data.keys():
        text_of_book = data[key]
       
        pattern = r'^[¡]*'
        
        new_sentence = []
        for sentence in text_of_book:
            sentence = sentence.strip('\n')
            sentence = re.sub(pattern, '', sentence)
            new_sentence.append(sentence)
            
        a = sent_tokenize(str(new_sentence))
        data_into_sentences[key] = a


    return data_into_sentences

In [6]:
# data processing: we need each novel to be tokenized into sentences, not a whole novel, not tokens.
### 此方法断句避免了断开Mr. Weasley的问题，可用。 当前主函数调用的是该函数。###
def pre_processing(data):
    
    data_into_sentences = {}
    
    for key in data.keys():
        text_of_book = data[key]
       
        pattern = r'^[¡]*'
        
        all_paragraph = []
        for para in text_of_book:
            para = para.strip('\n')
            para = re.sub(pattern, '', para)
            all_paragraph.append(para)
        
        all_sentence = []
        for i in all_paragraph:
            for j in sent_tokenize(i):
                all_sentence.append(j)

        data_into_sentences[key] = all_sentence


    return data_into_sentences

In [14]:
# Run main function and generate valid data
valid_data = main()

In [15]:
# See what valid_data looks like and check whether there is problem
print(valid_data.keys())

dict_keys(['Harry Potter and the Deathly Hallows', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Sorcerer Stone', 'Harry Potter and The Half-Blood Prince', 'Harry Potter and The Chamber Of Secrets', 'Harry Potter and the Goblet of Fire'])


In [8]:
print(valid_data['Harry Potter and the Sorcerer Stone'][0])

CHAPTER ONE


In [9]:
print(valid_data['Harry Potter and the Sorcerer Stone'][1])

THE BOY WHO LIVED


In [10]:
print(valid_data['Harry Potter and the Sorcerer Stone'][2])

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.


In [11]:
print(valid_data['Harry Potter and the Sorcerer Stone'][3])

They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.


In [12]:
# Make the valid_data into content_json format as required # almost same as Janine's code
content_json = []
verse = 0
#book1
for sentence in valid_data['Harry Potter and the Sorcerer Stone']:
    verse = verse + 1
    chunk = {"book_id":1}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)
#book2
for sentence in valid_data['Harry Potter and The Chamber Of Secrets']:
    verse = verse+1
    chunk = {"book_id":2}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)
#book3
for sentence in valid_data['Harry Potter and the Prisoner of Azkaban']:
    verse = verse+1
    chunk = {"book_id":3}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)
#book4
for sentence in valid_data['Harry Potter and the Goblet of Fire']:
    verse = verse+1
    chunk = {"book_id":4}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)
#book5
for sentence in valid_data['Harry Potter and the Order of the Phoenix']:
    verse = verse+1
    chunk = {"book_id":5}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)
#book6
for sentence in valid_data['Harry Potter and The Half-Blood Prince']:
    verse = verse+1
    chunk = {"book_id":6}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)
#book7
for sentence in valid_data['Harry Potter and the Deathly Hallows']:
    verse = verse+1
    chunk = {"book_id":7}
    chunk["text"] = sentence
    chunk["verse"] = verse
    content_json.append(chunk)

In [13]:
# See what the content_json looks like
print(type(content_json))
print('---')
print(content_json[:2])

<class 'list'>
---
[{'book_id': 1, 'text': 'CHAPTER ONE', 'verse': 1}, {'book_id': 1, 'text': 'THE BOY WHO LIVED', 'verse': 2}]


In [14]:
# Create final_data for next step, namely find subject, verb, and object
final_data = []
for i in range(len(content_json)):
    final_data.append(content_json[i]["text"])

In [15]:
# See what the final_data looks like
print(final_data[50])

He'd forgotten all about the people in cloaks until he passed a group of them next to the baker's.


In [16]:
# Find subject + verb + object!!! 
### Time complexity is n. time: 18min

### Result using data produced by preprocessing（）:
# subject + verb: 24057
# subject + verb + object (helper1 = helper2 = 1): 3222
# subject + verb + object (helper1 >= helper2 >= 1): 4582
###

### Result using data produced by pre_processing（）:
# subject + verb + object (helper1 = helper2 = 1): 2994
# 其他还没试
####

### Corner case （可以先不管）
'''
Mrs. Potter was Mrs. Dursley's sister, but they hadn't met for several years; 
in fact, Mrs. Dursley pretended she didn't have a sister, because her sister and
her good-for-nothing husband were as unDursleyish as it was possible to be.
'''


sub_verb_obj = []

def token_is_subject_with_action(token):
    nsubj = token.dep_ == 'nsubj'
    head_verb = token.head.pos_ == 'VERB'
    person = token.ent_type_ == 'PERSON'
    return nsubj and head_verb and person

def token_is_dobject(token):
    dobj = token.dep_ == 'dobj'
    person = token.ent_type_ == 'PERSON'    
    return  dobj and person                 

def token_is_pobject(token):
    pobj = token.dep_ == 'pobj'
    person = token.ent_type_ == 'PERSON'    
    return  pobj and person


for i in range(len(final_data)):
    doc = nlp(final_data[i])
    helper_nsubj, helper_obj = 0, 0 
    
    for token in doc:
        if token_is_subject_with_action(token):
            helper_nsubj += 1
            token_nsubj = token
            
        elif token_is_dobject(token) or token_is_pobject(token):
            helper_obj += 1
            token_obj = token
    
    # 此处可以有多种修改模式，当前条件是一句话里有一个主语和一个宾语
    # 可以改成一句话里至少有一个主语和一个宾语
    # 也可以改成两个helper相加为2，等多种方法，比较哪种方法比较好
    if helper_nsubj == 1 and helper_obj == 1:                    
        span = doc[token_nsubj.head.left_edge.i:token_nsubj.head.right_edge.i+1]
        # print(span)
        data = dict(
                    name = token_nsubj.orth_,
                    span = span.text,
                    verb = token_nsubj.head.lower_,
                    log_prob = token_nsubj.head.prob,
                    obj = token_obj.orth_
                    )

        sub_verb_obj.append(data)
            

print(len(sub_verb_obj)) 

2994


In [17]:
# See what the result looks like
print(sub_verb_obj[0])

{'name': 'Voldemort', 'span': "that last night Voldemort turned up in Godric's Hollow", 'verb': 'turned', 'log_prob': -9.319558143615723, 'obj': 'Hollow'}


In [18]:
print(sub_verb_obj[15])

{'name': 'Petunia', 'span': 'After asking Harry furiously if he knew the man, Aunt Petunia had rushed them out of the shop without buying anything.', 'verb': 'rushed', 'log_prob': -11.93223762512207, 'obj': 'Harry'}


In [20]:
print(sub_verb_obj[1])

{'name': 'Dumbledore', 'span': 'I would trust Hagrid with my life," said Dumbledore.', 'verb': 'said', 'log_prob': -7.258025169372559, 'obj': 'Hagrid'}
